The output from the Skyline document (if using a novel document and not the preformed human/conserved document) does not include Peptide Notes with the corresponding histone mark (e.g. H3 K27me1). I want to use the hard-coded table of histone marks and their respective mass shifts provided in the complementary `generate_hptms.ipynb` script to reverse-engineer the Skyline Peptide Modified Sequence back into biological histone mark notation (e.g. `PEK[+42.04695]TIDER` would become `H3 K3me1`).


input: a CSV with Peptide Modified Sequences from `Skyline>File>Export>Custom Report...` formats

output: a new column with the Peptide Note using "histone mod" language (H3K27me1, H3K4ac, etc)

In [56]:
import sys
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import re
import numpy as np
from Bio import SeqIO
from pyteomics import fasta, parser, mass, achrom, electrochem, auxiliary
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import itertools

sys.stdout.write("Imported required packages successfully.\n")

# set the master table for modification mass shifts
MZSHIFT_DICT = {'[nPR]':'[+56]',
                '[AC]': '[+42]',
                '[PR]': '[+56]',
                '[ME1]': '[+70]',
                '[ME2]': '[+28]',
                '[ME3]': '[+42]',
                '[PH]': '[+80]',
                '[nME3PR]': '[+98.1]',
                '[nME2PR]': '[+84.1]',
                '[nME1PR]': '[+126.1]',
                '[nPR2]': '[+112.1]',
                '[nAC]': '[+98]',
                '[nACPR]': '[+84.1]'}


##
## read input files: FASTA and Skyline Export Report with Peptide Modified Sequences
##

# read in FASTA
fasta_file = os.path.join(os.getcwd(), 
                          "../../collab_greer/data/uniprot-dicty_histones_7entries_acc20201231.fasta")

protein_df = pd.DataFrame()  # Initialize a dataframe to store results
for protein in tqdm(SeqIO.parse(fasta_file, "fasta")):
    
    protein_sequence = str(protein.seq).upper()
    
    # cleave initial "start" methionine if present
    if protein_sequence[0] == "M":
        protein_sequence = protein_sequence[1:]

    # add this protein to the dataframe
    new_df = pd.DataFrame({'protein': protein.id,
                           'protein_sequence': protein_sequence}, index=[0])
    
    protein_df = protein_df.append(new_df)
protein_df = protein_df.drop_duplicates()


# read in Skyline Export Report with Peptide Modified Sequences
skyline_df = pd.read_csv(os.path.join(os.getcwd(), 
                                      "../../collab_greer/data/greer_onlyhistlibrary_groupcomparison_mound-v-vegetative.csv"))

##
## "decode" modified peptide sequences to biological histone marks
##


# remove propionylations ([+56], [+112.1]) which aren't biologically relevant here
skyline_df['new_pep_seq'] = skyline_df['Peptide Modified Sequence']
skyline_df['new_pep_seq'] = skyline_df['new_pep_seq'].str.replace(r'\[\+56\]', '')
skyline_df['new_pep_seq'] = skyline_df['new_pep_seq'].str.replace(r'\[\+112.1\]', '')

# decode each modified peptide sequence to its modified residue number and mod type
decode_df = pd.DataFrame()  # Initialize a dataframe to store results
for index, row in skyline_df.iterrows():
    peptide = row['Peptide']
    mod_seq = row['new_pep_seq']
    fc = row['Fold Change Result']
    pval = row['Adjusted P-Value']
    
    # find all protein matches for the peptide in case there are duplicate/non-unique peptides
    protein_match = list(protein_df[protein_df['protein_sequence'].str.contains(peptide)]['protein'])
    
    # map each peptide to its residue position in the protein and change mass shift to modification
    for protein in protein_match:
        sequence = protein_df[protein_df['protein'] == protein]['protein_sequence'][0]
        aa_index = sequence.find(peptide)  # get the amino acid index position
        
        # split the peptide sequence into residues including any [+nn] mod shift
        residue_list = re.sub( r"([A-Z])", r" \1", mod_seq).split()

        # build the [Residue][Index Position][modification] histone mark
        histone_mod = ''
        for i in range(len(residue_list)): 
            if '[' in residue_list[i]:
                aa_pos = aa_index + i + 1  # have to +1 for indexing
                new_mod = residue_list[i][0] + str(aa_pos) + residue_list[i][1:]
                histone_mod = histone_mod + new_mod
        
        # add this protein/peptide to the dataframe
        new_df = pd.DataFrame({'Protein Name': protein,
                               'Peptide Sequence': peptide,
                               'Peptide Modified Sequence': row['Peptide Modified Sequence'],
                               'histone mark': histone_mod,
                               'Fold Change Result': fc,
                               'Adjusted P-Value': pval}, index=[0])
        decode_df = decode_df.append(new_df)
        
decode_df = decode_df.drop_duplicates()

decode_df = decode_df.groupby(['Peptide Modified Sequence', 'histone mark', 
                               'Fold Change Result', 'Adjusted P-Value'])['Protein Name'].apply(lambda x: ','.join(x)).reset_index()

decode_df.to_csv("D:/Penn/proj/collab_greer/data/greer_onlyhistlibrary_groupcomparison_mound-v-vegetative_decoded.csv", 
                        index=False)

decode_df.tail()

7it [00:00, 1403.92it/s]

Imported required packages successfully.


,Peptide Modified Sequence,histone mark,Fold Change Result,Adjusted P-Value,Protein Name
102,VTIMVK[+56]DIQLAR,,0.94 (95% CI:0.07 to 12.39),0.9813,sp|Q55BN9|H33B_DICDI
103,V[+56]KPVPK[+56]STK[+56]AGLIFPVGR,,0.95 (95% CI:0 to 205817.11),0.9910,sp|Q54LA5|H2AZ_DICDI
104,V[+56]TIMPK[+56]DIHLAR,,5939.19 (95% CI:197 to 179052.79),0.0386,sp|O15819|H33A_DICDI
105,V[+56]TIMPK[+70]DIHLAR,K125[+70],3.7 (95% CI:0.66 to 20.69),0.3299,sp|O15819|H33A_DICDI
106,V[+56]TIMVK[+56]DIQLAR,,4.5 (95% CI:1.86 to 10.89),0.0526,sp|Q55BN9|H33B_DICDI
